<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Launch Sites Locations Analysis with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:
- **TASK 1:** Mark all launch sites on a map
- **TASK 2:** Mark the success/failed launches for each site on the map
- **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
!pip3 install folium
!pip3 install wget
!pip3 install pandas

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9687 sha256=dbc562510c7ca5ccf8c6fcf8ee77acd87a147ec596a5576a4b9f69b074e4a213
  Stored in directory: /root/.cache/pip/wheels/8a/b8/04/0c88fb22489b0c049bee4e977c5689c7fe597d6c4b0e7d0b6a
Successfully built wget


In [2]:
import folium
import wget
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb)


## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site. 


In [4]:
# Download and read the `spacex_launch_geo.csv`
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df=pd.read_csv(spacex_csv_file)

100% [................................................................................] 7710 / 7710

Now, you can take a look at what are the coordinates for each site.


In [5]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [6]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example, 


In [7]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle. 


Now, let's add a circle for each launch site in data frame `launch_sites`


_TODO:_  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [9]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label

import folium
from folium import DivIcon

# Define a central location for the map
nasa_coordinate = [37.7749, -122.4194]  # Replace with a more relevant central coordinate

# Initialize the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Loop through each launch site and add Circle and Marker to the map
for a, row in launch_sites_df.iterrows():
    coordinate = [row['Lat'], row['Long']]
    launch_site_name = row['Launch Site']
    
    # Add Circle
    folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(launch_site_name)).add_to(site_map)
    
    # Add Marker
    folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20, 20), icon_anchor=(0, 0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % launch_site_name)).add_to(site_map)

# Display the map
site_map


The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:
- Are all launch sites in proximity to the Equator line?
- Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


# Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [10]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records. 
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [11]:
marker_cluster = MarkerCluster()

# Loop through each launch record and add a marker to the map
for _, row in spacex_df.iterrows():
    coordinate = [row['Lat'], row['Long']]
    launch_status = 'Success' if row['class'] == 1 else 'Failure'
    color = 'green' if row['class'] == 1 else 'red'
    
    # Add Marker with different colors based on launch status
    folium.map.Marker(
        coordinate,
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12; color:%s;"><b>%s</b></div>' % (color, launch_status)
        )
    ).add_to(site_map)

# Display the map with launch markers
site_map

_TODO:_ Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value


In [15]:
# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red

# Merge the launch_sites_df with the spacex_df to get the 'class' column
launch_sites_df = launch_sites_df.merge(spacex_df[['Launch Site', 'class']], on='Launch Site', how='left')

# Add a new column 'marker_color' based on the 'class' value
launch_sites_df['marker_color'] = launch_sites_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

launch_sites_df.head()

,Launch Site,Lat,Long,class_x,marker_color,class_y,class
0,CCAFS LC-40,28.562302,-80.577356,0,red,0,0
1,CCAFS LC-40,28.562302,-80.577356,0,red,0,0
2,CCAFS LC-40,28.562302,-80.577356,0,red,0,0
3,CCAFS LC-40,28.562302,-80.577356,0,red,0,0
4,CCAFS LC-40,28.562302,-80.577356,0,red,0,0


In [16]:
# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


_TODO:_ For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [17]:
import folium
from folium.plugins import MarkerCluster

# Initialize the map
nasa_coordinate = [37.7749, -122.4194]  # Replace with a relevant central coordinate
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Create MarkerCluster object
marker_cluster = MarkerCluster().add_to(site_map)

# Loop through each launch record in spacex_df
for a, record in spacex_df.iterrows():
    coordinate = [record['Lat'], record['Long']]
    launch_status = 'Success' if record['class'] == 1 else 'Failure'
    color = 'green' if record['class'] == 1 else 'red'
    
    # Create and add a Marker to the marker_cluster
    marker = folium.Marker(
        location=coordinate,
        icon=folium.Icon(color=color),
        popup=folium.Popup(f"Launch Site: {record['Launch Site']}<br>Status: {launch_status}", parse_html=True)
    )
    
    marker_cluster.add_child(marker)

# Display the map with marker cluster
site_map


Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [18]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


You can calculate the distance between two points on the map based on their `Lat` and `Long` values using the following method:


In [19]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

_TODO:_ Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [27]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Initialize the map
nasa_coordinate = [37.7749, -122.4194]  # Replace with a relevant central coordinate
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

site_map.add_child(mouse_position)

# Create MarkerCluster object
marker_cluster = MarkerCluster().add_to(site_map)

# Function to calculate the distance between two points (latitude, longitude)
def calculate_distance(coord1, coord2):
    return geodesic(coord1, coord2).km  # Returns distance in kilometers

# For each row in spacex_df, create a marker and add it to marker_cluster
for _, record in spacex_df.iterrows():
    coordinate = [record['Lat'], record['Long']]
    launch_status = 'Success' if record['class'] == 1 else 'Failure'
    color = 'green' if record['class'] == 1 else 'red'
    
    # Create and add a Marker to the marker_cluster
    marker = folium.Marker(
        location=coordinate,
        icon=folium.Icon(color=color),
        popup=folium.Popup(f"Launch Site: {record['Launch Site']}<br>Status: {launch_status}", parse_html=True)
    )
    
    marker_cluster.add_child(marker)

# Function to add a marker for the closest coastline point and calculate the distance
def on_map_click(e):
    # Get the coordinates where the user clicks (closest coastline)
    coastline_coords = [e.latlng.lat, e.latlng.lng]
    
    # Mark the point on the map
    folium.Marker(
        location=coastline_coords,
        icon=folium.Icon(color='blue', icon='info-sign'),
        popup="Closest Coastline Point"
    ).add_to(site_map)
    
    # Calculate and print the distance from the launch site (e.g., first row for demonstration)
    launch_site_coords = [spacex_df.iloc[0]['Lat'], spacex_df.iloc[0]['Long']]  # Replace with the desired launch site
    distance = calculate_distance(launch_site_coords, coastline_coords)
    
    print(f"Distance between the coastline and the launch site: {distance:.2f} km")

# Display the map
site_map


_TODO:_ After obtained its coordinate, create a `folium.Marker` to show the distance


In [ ]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
# distance_marker = folium.Marker(
#    coordinate,
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
#        )
#    )

_TODO:_ Draw a `PolyLine` between a launch site to the selected coastline point


In [28]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)

# Initialize the map
nasa_coordinate = [37.7749, -122.4194]  # Replace with a relevant central coordinate
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Add MousePosition plugin to display latitude and longitude of cursor
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)

# Create MarkerCluster object
marker_cluster = MarkerCluster().add_to(site_map)

# Function to calculate the distance between two points (latitude, longitude)
def calculate_distance(coord1, coord2):
    return geodesic(coord1, coord2).km  # Returns distance in kilometers

# For each row in spacex_df, create a marker and add it to marker_cluster
for _, record in spacex_df.iterrows():
    coordinate = [record['Lat'], record['Long']]
    launch_status = 'Success' if record['class'] == 1 else 'Failure'
    color = 'green' if record['class'] == 1 else 'red'
    
    # Create and add a Marker to the marker_cluster
    marker = folium.Marker(
        location=coordinate,
        icon=folium.Icon(color=color),
        popup=folium.Popup(f"Launch Site: {record['Launch Site']}<br>Status: {launch_status}", parse_html=True)
    )
    
    marker_cluster.add_child(marker)

# Function to add a marker for the closest coastline point, calculate the distance, and draw a PolyLine
def on_map_click(e):
    # Get the coordinates where the user clicks (closest coastline)
    coastline_coords = [e.latlng.lat, e.latlng.lng]
    
    # Mark


NameError: name 'lines' is not defined

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


_TODO:_ Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [30]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site


In [29]:
# Example coordinates for cities, railways, and highways (replace with actual data or sources)
cities = {
    'City1': [37.7749, -122.4194],  # San Francisco (example)
    'City2': [34.0522, -118.2437],  # Los Angeles (example)
    'City3': [40.7128, -74.0060],   # New York (example)
}

# Initialize the map
nasa_coordinate = [37.7749, -122.4194]  # Center map (replace with a relevant central coordinate)
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Add MousePosition plugin to display latitude and longitude of cursor
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)

# Function to calculate the distance between two points (latitude, longitude)
def calculate_distance(coord1, coord2):
    return geodesic(coord1, coord2).km  # Returns distance in kilometers

# Find the closest city to a given location
def find_closest_feature(feature_coords, launch_site_coords):
    min_distance = float('inf')
    closest_feature = None

    for feature, coords in feature_coords.items():
        distance = calculate_distance(launch_site_coords, coords)
        if distance < min_distance:
            min_distance = distance
            closest_feature = feature
    
    return closest_feature, min_distance

# Example: Using the first launch site in spacex_df
launch_site_coords = [spacex_df.iloc[0]['Lat'], spacex_df.iloc[0]['Long']]  # Replace with a specific launch site if needed

# Find the closest city to the launch site
closest_city, distance_to_city = find_closest_feature(cities, launch_site_coords)

# Create and add a marker for the closest city
closest_city_coords = cities[closest_city]
folium.Marker(
    location=closest_city_coords,
    icon=folium.Icon(color='blue', icon='info-sign'),
    popup=f"{closest_city}<br>Distance: {distance_to_city:.2f} km"
).add_to(site_map)

# Create and add a marker for the launch site
folium.Marker(
    location=launch_site_coords,
    icon=folium.Icon(color='green', icon='cloud'),
    popup="Launch Site"
).add_to(site_map)

# Draw a line between the launch site and the closest city
folium.PolyLine(
    locations=[launch_site_coords, closest_city_coords],
    color='blue',
    weight=3,
    opacity=0.6
).add_to(site_map)

# Display the map
site_map

NameError: name 'geodesic' is not defined

After you plot distance lines to the proximities, you can answer the following questions easily:
- Are launch sites in close proximity to railways?
- Are launch sites in close proximity to highways?
- Are launch sites in close proximity to coastline?
- Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Yan Luo](https://www.linkedin.com/in/yan-luo-96288783/)


### Other Contributors


Joseph Santarcangelo


## Change Log


|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2021-05-26|1.0|Yan|Created the initial version|


Copyright © 2021 IBM Corporation. All rights reserved.
